# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/electricity/2_electricity_feature_pipeline.ipynb)

## 🗒️ This notebook is divided into 2 sections:
1. Parse Data.
2. Insert new data into the Feature Store.

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install -U hopsworks --quiet

# Hosted notebook environments may not have the local features package
import os 

def need_download_modules():
    if 'google.colab' in str(get_ipython()):
        return True
    if 'HOPSWORKS_PROJECT_ID' in os.environ:
        return True
    return False

if need_download_modules():
    print("⚙️ Downloading modules...")
    os.system('mkdir -p features')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/electricity/features/electricity_prices.py')
    os.system('cd features && wget https://raw.githubusercontent.com/logicalclocks/hopsworks-tutorials/master/advanced_tutorials/electricity/features/meteorological_measurements.py')
    print('✅ Done!')
else:
    print("Local environment")

In [ ]:
import requests
import pandas as pd

from features import electricity_prices, meteorological_measurements

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

---

## <span style="color:#ff5f27;"> 🪄 Parsing new data</span>

### <span style="color:#ff5f27;"> 🌤 Meteorological measurements from SMHI</span>

In [ ]:
meteorological_measurements_df = meteorological_measurements.fetch_smhi_measurements(
    historical_data=False,
)
meteorological_measurements_df

### <span style="color:#ff5f27;">💸 Electricity prices per day from NORD POOL</span>

In [ ]:
electricity_prices_df = electricity_prices.fetch_electricity_prices(
    historical=False,
)
electricity_prices_df

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

In [ ]:
meteorological_measurements_fg = fs.get_or_create_feature_group(
    name="meteorological_measurements",
    version=1,
)

electricity_prices_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
)

---

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
meteorological_measurements_fg.insert(meteorological_measurements_df)

In [ ]:
electricity_prices_fg.insert(
    electricity_prices_df, 
    write_options={"wait_for_job": True},
)

---

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline </span>

In the next notebook, you will create a feature view, training dataset, train a model and save it in model registry.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/electricity/3_electricity_training_pipeline.ipynb)